### Prerequsites:
##### 1. Add Input -> Competition Datasets -> search "Deep Learning Spring 2025: CIFAR 10 classification" -> Click + icon to add the competition's datasets
##### 2. Sign in and set accelerator in session options to GPU P100


#### 1. Check if the competition files were downloaded as input properly, expect the printed path of the downloaded files 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#### 2. Clone github repo for access to functions

In [ ]:
!git clone https://github.com/aaronbengochea/cifar10-research.git
%cd cifar10-research

#### 3. Import functions from the cloned github repo

In [ ]:
import torch
import torch.optim as optim
from torchsummary import summary
from resnet import create_basicblock_model
from training import main as train
from inference import main as inference

#### 4. Set model parameters

In [ ]:
# Model parameters
MODEL_NAME = 'ResNet40_v2'
NUM_STARTING_INPUT_CHANNELS = 3
NUM_BLOCKS_PER_LAYER = [5, 7, 4, 3]
NUM_CHANNELS_PER_LAYER = [32, 64, 128, 256]
KERNEL_SIZE_PER_LAYER = [3, 3, 3, 3]
SKIP_KERNEL_SIZE_PER_LAYER = [1, 1, 1, 1]


# Training parameters
EPOCHS = 320
TRAIN_BATCH_SIZE = 128
TEST_BATCH_SIZE = 100
AUGMENT = True

#### 5. Create the model, check parameter count and generate model summary

In [ ]:
model = create_basicblock_model(
    name = MODEL_NAME,
    starting_input_channels = NUM_STARTING_INPUT_CHANNELS,
    blocks_per_layer = NUM_BLOCKS_PER_LAYER,
    channels_per_layer = NUM_CHANNELS_PER_LAYER,
    kernels_per_layer = KERNEL_SIZE_PER_LAYER,
    skip_kernels_per_layer = SKIP_KERNEL_SIZE_PER_LAYER
    
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print('Total model parameters:', sum(p.numel() for p in model.parameters() if p.requires_grad))
summary(model, (3, 32, 32))

#### 6. Define the optimizer of choice (only choose one, comment the rest)

In [ ]:
OPTIMIZER = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
# OPTIMIZER = optim.Adam(model.parameters(), lr=0.1, weight_decay=1e-4)

#### 7. Define the schedular of choice (only choose one, comment the rest)

In [ ]:
SCHEDULER = optim.lr_scheduler.CosineAnnealingLR(OPTIMIZER, T_max=EPOCHS)
# SCHEDULER = optim.lr_scheduler.ReduceLROnPlateau(OPTIMIZER, mode='max', factor=0.50, patience=3)

#### 8. Train the model
**Model Checkpoints**
- Each time we achieve a new accuracy milestone on the CIFAR-10 labeled testset, we save the milestone model
- Model checkpoint directory: /kaggle/working/model_checkpoint_history

**Model Training/Test Performance History**
- We save our models performance metrics on each epoch cycle
- The metrics include Avg train/test loss, train/test accuracy, and learning rate
- Model performance directory: /kaggle/working/training_history

In [ ]:
train(
    model=model,
    epochs=EPOCHS,
    train_batch_size=TRAIN_BATCH_SIZE,
    test_batch_size=TEST_BATCH_SIZE,
    augment=AUGMENT,
    optimizer=OPTIMIZER,
    scheduler=SCHEDULER
)

#### 9. Perform inference on the competition hidden testset
- Inference is performed on all models saved as checkpoints, this allows us to test different model checkpoints vs the hidden testset
- Model inference directory: /kaggle/working/predictions_history

In [ ]:
inference()